# Ruijie V2 Web Portal API

本笔记本致力于逆向燕山大学使用的锐捷V2版本Web登陆系统，将只能在浏览器中使用的Web界面转换为一个命令行登陆器。

这是草稿板，自己拿着玩去吧

需要安装的软件包： `requests pysocks pycryptodome`

笔记本还依赖 `ysu_login.py` 作为统一身份认证登录的提供者

## 导入软件包

In [1]:
import requests

import time
import functools
import inspect
from urllib.parse import urlparse, parse_qs
import ysu_login

## 配置使用的http客户端以及一些杂七杂八的东西

**如果你要在本机上使用，记得把 `proxies` 设置为空，否则会失败**，这里配置 `proxies` 仅是为了防止开发期间我的电脑频繁登入登出

In [2]:
# Set proxy
# NOTICE: remove this to actually do things in this machine
proxies = {
    "http": "socks5://127.0.0.1:10800",
    "https": "socks5://127.0.0.1:10800"
}

# Generate request client
client = requests.Session()
client.proxies.update(proxies)
# UA
client.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
})

## 一些比较有用的装饰器

我很喜欢它们

In [3]:
# Func tools
def unwrap_response(json=False):
    def _unwrap_response(func):
        """Decorator to unwrap the response from a function.

        If the function returns a requests.Response object, it will check the status code.
        If param json is set to True, it will jsonlize the response text, check its `code` property and return its `data` property.
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            response = func(*args, **kwargs)
            if isinstance(response, requests.Response):
                response.raise_for_status()  # Raise an error for bad responses
                if json:
                    data = response.json()
                    if data.get("code") == 200:
                        return data.get("data")
                    raise ValueError(f"API error: {data.get('message')}")
                return response.json()
            return response
        return wrapper
    return _unwrap_response

In [4]:
# GetCurrentNode API
# This API is intend to check the login progress after each requests.
# Therefore it is implemented in the form of decorator.

def get_current_node(flowKey='portal_auth'):
    def _get_current_node(func):
        """Decorator to get the current node of the login flow."""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            response = func(*args, **kwargs)
            # Get the func's first arg
            sig = inspect.signature(func)
            bound_args = sig.bind(*args, **kwargs)
            bound_args.apply_defaults()
            session_info = bound_args.arguments['session_info']
            node_url = "https://auth1.ysu.edu.cn/eportal/workFlow/getCurrentNode"
            node_resp = client.post(
                node_url,
                json={
                    "sessionId": session_info['sessionId'],
                    "flowKey": flowKey
                },
                proxies=proxies
            ).json()
            print(f"Current Node: {node_resp['data'].get('currentNodePath', 'Unknown')}")
            return response
        return wrapper
    return _get_current_node

## API

自己看函数签名和注释吧

In [5]:
# GetOnlineUserInfo
# Refer to `TECH.md` for the API details

@unwrap_response(json=True)
def get_online_user_info(session_id='114514'):
    timestamp = int(time.time() * 1000)
    return client.get(f"https://auth1.ysu.edu.cn/eportal/adaptor/getOnlineUserInfo?sessionId={session_id}&{timestamp}&version=this%20is%20a%20git-commit", proxies=proxies)

In [6]:
# Test

get_online_user_info()

{'portalOnlineUserInfo': {'userIndex': None,
  'result': 'fail',
  'message': 'dx.failed.user.offline',
  'keepaliveInterval': None,
  'maxLeavingTime': None,
  'maxFlow': None,
  'userName': None,
  'userId': None,
  'userIp': None,
  'userIpv6': None,
  'userMac': None,
  'webGatePort': None,
  'webGateIp': None,
  'service': None,
  'realServiceName': None,
  'apMac': None,
  'vlanId': None,
  'ssid': None,
  'accountInfo': None,
  'loginType': None,
  'utrustUrl': None,
  'userUrl': None,
  'pubMessage': None,
  'userGroup': None,
  'accountFee': None,
  'wlanAcName': None,
  'hasMabInfo': None,
  'isAlowMab': None,
  'userPackage': None,
  'selfUrl': None,
  'ballInfo': None,
  'notify': None,
  'resource': None,
  'offlineurl': None,
  'redirectUrl': 'http://123.123.123.123/',
  'announcement': None,
  'userNotice': None,
  'systemNotice': None,
  'portalIp': None,
  'portalUrl': None,
  'rghallUrl': None,
  'welcomeTip': None,
  'serviceList': None,
  'isSuccessService': None,
 

In [7]:
def redirect_to_portal(redirect_url = 'https://auth1.ysu.edu.cn/eportal/redirect.jsp?mode=history') -> dict:
    """Redirect to the portal URL, and return params that may help.
    For example: sessionId, userIp, userMac, etc.
    """
    resp = client.get(redirect_url, allow_redirects=True, proxies=proxies)
    if "location.href=" in resp.text:
        redirect_url_2 = resp.text.split("'")[1].split("'")[0]
        resp = client.get(redirect_url_2, allow_redirects=True, proxies=proxies)
    assert("portal-main" in resp.request.url), f"Not redirected to portal, check the network. Its URL should ends with 'portal-main' but actually {resp.request.url}"
    # Extract the request params from resp.request.url
    parsed_url = urlparse(resp.request.url)
    request_params = parse_qs(parsed_url.query)
    # Remove the values' list and keep only the first value
    request_params = {k: v[0] for k, v in request_params.items()}
    return request_params

# redirect_to_portal("http://123.123.123.123")

In [8]:
# QueryTerminalInfo API
# TODO

In [9]:

# SAM Login API
@get_current_node()
def sam_login(session_info: dict) -> dict:
    """Login to SAM with the provided session info."""
    session_id = session_info['sessionId']
    custom_page_id = session_info['customPageId']
    nas_ip = session_info['nasIp']
    user_ip = session_info['userIp']
    ssid = session_info['ssid']
    user_mac = session_info['userMac']
    sam_url = f"https://auth1.ysu.edu.cn/sam-sso/login?flowSessionId={session_id}&customPageId={custom_page_id}&preview=false&appType=normal&language=zh-CN&nasIp={nas_ip}&userIp={user_ip}&ssid={ssid}&userMac={user_mac}"
    resp = client.post(sam_url, json=session_info, proxies=proxies, allow_redirects=True)
    # print(resp.request.url)
    # print(resp.text)
    
    # Client redirect to YSU CAS Server
    cas_redirect_url = "https://auth1.ysu.edu.cn/sam-sso/clientredirect?client_name=sidadapter&service=https://auth1.ysu.edu.cn/portal/entry/pc/authenticate;flowParams=undefined;from="
    resp = client.get(cas_redirect_url, allow_redirects=True, proxies=proxies)
    # print(resp.request.url)
    # print(resp.text)
    assert "cer.ysu.edu.cn" in resp.request.url or "ticket=" in resp.request.url, f"Expect redirect to CAS server(cer.ysu.edu.cn) or successfully logged in, but actually {resp.request.url}"

# sam_login(session_info)

In [10]:
# ServiceSelection API

@get_current_node()
@unwrap_response(json=True)
def service_selection(session_info: dict) -> dict:
    """Select the service for the user."""
    service_url = "https://auth1.ysu.edu.cn/eportal/network/serviceSelection"
    resp = client.post(service_url, json={
        "sessionId": session_info['sessionId']
    }, proxies=proxies)
    return resp

# service_selection(session_info)

In [11]:
# ServiceLogin API

# @unwrap_response(json=True) # This API sometimes returns error even logged in actually. So disable it.
@get_current_node()
def service_login(session_info: dict, service="校园网") -> dict:
    """Login to the service with the provided session info."""
    service_url = "https://auth1.ysu.edu.cn/eportal/network/serviceLogin"
    resp = client.post(service_url, json={
        "sessionId": session_info['sessionId'],
        "service": service
    }, proxies=proxies)
    return resp.json()

# service_login(session_info)

In [12]:
# UserOnline API

@unwrap_response(json=True)
def user_online(session_info: dict) -> dict:
    """Check if the user is online."""
    online_url = "https://auth1.ysu.edu.cn/eportal/network/userOnline"
    resp = client.post(online_url, json={
        "sessionId": session_info['sessionId']
    }, proxies=proxies)
    return resp

# user_online(session_info)

In [13]:
# GetAccountInfo API

@unwrap_response(json=True)
def get_account_info(session_info: dict) -> dict:
    """Get account information."""
    account_url = "https://auth1.ysu.edu.cn/eportal/operator/getAccountInfo"
    resp = client.post(account_url, json={
        "sessionId": session_info['sessionId']
    }, proxies=proxies)
    return resp

# get_account_info(session_info)


In [14]:
# Offline API
# This API logout the user from the system
@unwrap_response(json=True)
def offline(session_info: dict) -> dict:
    """Logout the user."""
    offline_url = "https://auth1.ysu.edu.cn/eportal/network/offline"
    resp = client.post(offline_url, json={
        "sessionId": session_info['sessionId']
    }, proxies=proxies)
    return resp

# offline(session_info)

## 测试部分

这是真正实用的东西，你可以看到如何组合这些API来实现登录和登出，但是首先让我们输入你的统一身份认证用户名和密码

In [15]:
username = input("Username: ")
password = input("Password: ")

下面是登录逻辑

In [16]:
# Login logic1: 
# 1. GetOnlineUserInfo
user_info: dict = get_online_user_info()
redirect_url = user_info["portalOnlineUserInfo"].get("redirectUrl")

# 2. Redirect to portal to get session info
session_info = redirect_to_portal()
print(f"Session info: {session_info}")

assert redirect_url, "Should not logged in."

# 3. CAS Login

cas_client = ysu_login.YSULogin(username, password, session=client, proxies=proxies)
cas_result = cas_client.login()
assert cas_result, "CAS login failed, check your username and password."

# 4. SAM Login
sam_login(session_info)

# 5. Get available service list
print(service_selection(session_info))

# 6. Select service
service_login(session_info, "校园网")

# 7. Check user online status
print(user_online(session_info))

Session info: {'sessionId': 'e3c96dabbf78', 'userIp': '10.20.75.20', 'userMac': '525252525252', 'ssid': 'Ruijie', 'nasIp': '10.11.1.200', 'customPageId': 'defe500935844f5b9d03ecbabeaf69c8', 'mode': 'history', 'redirectUrl': 'http://2.2.2.2/', 'samPortalRedirectUrl': 'http://2.2.2.2', 'guestFlow': 'false'}
登录成功！正在跳转到: https://ehall.ysu.edu.cn/login?ticket=ST-4097045-YHClq3x5VUbfVApWKSyFdqKQpxQids7ciap9
确认登录成功。
Current Node: serviceSelection
Current Node: serviceSelection
[{'key': '中国电信', 'value': '中国电信'}, {'key': '中国联通', 'value': '中国联通'}, {'key': '校园网', 'value': '校园网'}, {'key': '中国移动', 'value': '中国移动'}]
Current Node: finish
{'sessionId': 'e3c96dabbf78', 'online': True, 'userIndex': '4448415398be831c0198c1cb189856cc', 'message': None}


下面是登出逻辑

In [ ]:
# Logout logic

# 1. GetOnlineUserInfo
user_info: dict = get_online_user_info()
redirect_url = user_info["portalOnlineUserInfo"].get("redirectUrl")
assert not redirect_url, "Should be logged in."

# 2. Redirect to portal to get session info
session_info = redirect_to_portal()
print(f"Session info: {session_info}")

# 3. Offline
offline(session_info)

# 4. Check user online status
print(user_online(session_info))

这里用 `GetOnlineUserInfo` API来检查登陆状态

In [ ]:
# Check user online status
u_info = get_online_user_info()
if user_id := u_info["portalOnlineUserInfo"].get("userId"):
    print(f"User {user_id} is online.")
else:
    print("User is offline.")

接下来可以把代码交给AI，让它给你写一个Haskell版本的程序出来了。
🤪🤪🤪